In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import datetime as dt
import pickle
from utils import fractional_years
from predcit import Predictor
pd.set_option("display.max_columns", None)

## Load Data

In [ ]:
data_path = '../data'
df_raw = pd.read_excel(f'{data_path}/inscriptos_2018_4_6.xlsx')

### Enrich with estimated bday from DNI

In [ ]:
model_info = {"location": "filesystem", "path": "./", "filename":"mega_spline_model.pickle"}
bday_predictor = Predictor(model_info)
df_raw['estimated_bday'] = np.array(bday_predictor.predict(df_raw.nrodocumento.values.reshape(-1, 1)))

In [ ]:
df_raw['estimated_age'] = (df_raw.FechaCpte - df_raw.estimated_bday).apply(lambda td: td.days/365.25)

### Enrich with foreigner from DNI

In [ ]:
df_raw['foreigner'] = df_raw.nrodocumento>90e6

### Enrich with course start date

In [ ]:
df_start_date = pd.read_csv(f'{data_path}/fecha_inicio_clases.csv')
df_start_date['end_inscr_date'] = df_start_date.fecha_fin_insripcion_estimada.apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
df_start_date['start_course_date'] = df_start_date.fecha_inicio_cursada.apply(lambda x: dt.datetime.strptime(x,'%d/%m/%Y'))
df_start_date.drop(columns=['fecha_fin_insripcion_estimada','fecha_inicio_cursada','Unnamed: 3'],inplace=True)

In [ ]:
df_raw = df_raw.merge(df_start_date, on='anioperi')
df_raw['start_course_to_inscr_days'] = ((df_raw['start_course_date']-df_raw['FechaCpte'])/(24*60*60*1e9)).astype(int)
df_raw['end_inscr_to_inscr_days'] = ((df_raw['end_inscr_date']-df_raw['FechaCpte'])/(24*60*60*1e9)).astype(int)
df_raw.drop(columns=['end_inscr_date','start_course_date'],inplace=True)

In [ ]:
df_raw

### Enrich with history
Past churns, past completed courses, proff historic churn


In [ ]:
df_raw[df_raw.Anio_id==2022].bajadeasistencia.value_counts()

In [ ]:
df_raw['year'] = df_raw['anioperi'].apply(lambda x: int(x.split('-')[0]))
df_raw['period'] = df_raw['anioperi'].apply(lambda x: int(x.split('-')[1]))
df_enr = df_raw.merge(df_raw,on='nrodocumento')

In [ ]:
c1 = (df_enr['year_x']==df_enr['year_y'])&(df_enr['period_x']>df_enr['period_y'])
c2 = df_enr['year_x']>df_enr['year_y']
df_enr = df_enr[c1|c2]

In [ ]:
df_enr.tail()#[df_enr.nrodocumento==36406220.0]

In [ ]:
df_enr[df_enr.nrodocumento==42496842.0]

### Enrich with convenio

In [ ]:
df_raw['con_convenio'] = df_raw.convenio!=df_raw.convenio.value_counts().index[0]

### Drop Incomplete semester

In [ ]:
df_raw = df_raw[df_raw['Anio_id']<2022]

### Drop unused columns

In [ ]:
drop_cols_1 = ["anioperi","cursointerno_id","FechaCpte","nrodocumento","Alumno_id","Sede_id","nombre","Anio_id",
"descnivel","cursodenivel","esidiomaonline","descsedefis","pais_id","nombpais","provincia_id","nombpcia","localidad_ubi","FechaNacimiento"]

In [ ]:
drop_cols_2 = ["edad",  # dirty column replaced by estimated_age  
               "estimated_bday", # estimated_age has the relevant info
               "nivel_id",  # Combination of "idioma_id","grado","TipoDeNivel"
              ]

In [ ]:
drop_cols_1.extend(drop_cols_2)

### Columns with high cardinality

In [ ]:
high_cardinality_cols = ["convenio","esidiomaic","nombprof"]

In [ ]:
df = df_raw.drop(columns=drop_cols_1)

### Try with LightGBM - NO One Hot Encoding

In [ ]:
# Set type of categorical variables so LGBM can predict
is_object = df.dtypes=='object'
for feat in is_object[is_object==True].index.values:
    df[feat] = df[feat].astype('category')

In [ ]:
# Split Train and Test
label = 'bajadeasistencia'
y = df[label].astype('int')
X = df.drop(columns=[label])
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.4)

In [ ]:
# Set Hyperparameters and train
from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV

lgbm = LGBMClassifier(min_data_in_leaf=10,max_depth=2,num_leaves=4,verbose_eval=1)
#cal_lgbm = CalibratedClassifierCV(lgbm, cv=10, method='isotonic')
cal_lgbm = CalibratedClassifierCV(lgbm, cv=5, method='sigmoid')
lgbm.fit(X_train, y_train)
cal_lgbm.fit(X_train, y_train)

In [ ]:
# Predict on test and train data
y_pred_test_proba = lgbm.predict_proba(X_test)[:,1]
y_pred_test_proba_cal = cal_lgbm.predict_proba(X_test)[:,1]
y_pred_train_proba = lgbm.predict_proba(X_train)[:,1]
y_pred_train_proba_cal = cal_lgbm.predict_proba(X_train)[:,1]

In [ ]:
# Measure ROC AUC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, plot_roc_curve
metrics = {'roc_auc_score': roc_auc_score,}

for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
# Get Feature Importance
feature_importance = {f:i for f, i in zip(lgbm.feature_name_, lgbm.feature_importances_)}
feature_importance = {f:i/sum(i for i in feature_importance.values()) for f,i in feature_importance.items()}
feature_importance = dict(sorted(feature_importance.items(), key=lambda item: item[1], reverse=True))
feature_importance

In [ ]:
# Plot Roc curve
%matplotlib inline
%matplotlib notebook
plot_roc_curve(lgbm, X_test, y_test, color='red')
plt.plot([0,1],[0,1],label='random',color='black')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Calibrate
%matplotlib inline
%matplotlib notebook
plt.hist(y_pred_test_proba,density=True)
plt.hist(y_pred_train_proba,density=True, alpha=0.25)
plt.hist(y_pred_test_proba_cal,density=True, alpha=0.25)
plt.hist(y_pred_train_proba_cal,density=True, alpha=0.25)
plt.show()

In [ ]:
%matplotlib inline
%matplotlib notebook
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(y_train,y_pred_train_proba_cal,n_bins=20, strategy='uniform')
max_proba = np.max([prob_pred.max(),prob_true.max()])

plt.plot(prob_pred, prob_true,'-o',label='calibrated')
prob_true, prob_pred = calibration_curve(y_train,y_pred_train_proba,n_bins=20,strategy='uniform')
max_proba = np.max([prob_pred.max(),prob_true.max(),max_proba])

plt.plot(prob_pred, prob_true,'-o',label='uncalibrated')
plt.plot([0,max_proba],[0,max_proba],'--',label='random',color='black')
plt.grid()
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
%matplotlib notebook
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(y_test,y_pred_test_proba_cal,n_bins=12,strategy='uniform')
max_proba = np.max([prob_pred.max(),prob_true.max()])
plt.plot(prob_pred, prob_true,'-o',label='calibrated')
prob_true, prob_pred = calibration_curve(y_test,y_pred_test_proba,n_bins=12,strategy='uniform')
max_proba = np.max([prob_pred.max(),prob_true.max(),max_proba])
plt.plot(prob_pred, prob_true,'-o',label='uncalibrated')
plt.plot([0,max_proba],[0,max_proba],'--',label='random',color='black')
plt.grid()
plt.legend()
plt.show()

### Try with other Models:
### We need to one-hot-encode columns

In [ ]:
df_ohe = df.drop(columns=high_cardinality_cols)

In [ ]:
df_ohe['is_male'] = df_ohe.Sexo == 'M'
df_ohe = df_ohe.drop(columns=['Sexo'])

In [ ]:
## DO ONE HOT ENCODE FOR COLUMNS columns_ohe
columns_ohe = ["Descripcion","Periodo_id","idioma_id","grado","deschora","TipoDeNivel","Cursada_id"]
binary_ohe = ["foreigner","con_convenio","is_male"]

In [ ]:
for attr in binary_ohe: # avoid creating 2 columns for binary cases
    df_ohe[attr] = df_ohe[attr].apply(lambda x: 1 if x else 0)

In [ ]:
df_ohe = pd.get_dummies(df_ohe,columns=columns_ohe,prefix=columns_ohe)

### Train/Test split

In [ ]:
label = 'bajadeasistencia'
y = df_ohe[label].astype('int')
X = df_ohe.drop(columns=[label])
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

### Normalize age

In [ ]:
attr = 'estimated_age'
age_mean, age_std = X_train[attr].mean(), X_train[attr].std()
X_train[attr] = (X_train[attr]-age_mean)/age_std
X_test[attr] = (X_test[attr]-age_mean)/age_std

### Fit model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier

log_reg = sklearn.linear_model.LogisticRegression(solver = 'liblinear', max_iter=500, penalty='l1')
random_forest = RandomForestClassifier(min_samples_leaf=5, max_depth=10)
xgmodel = XGBClassifier(eta=0.3, gamma=0.2, max_depth=5, subsample=0.5,reg_alpha=10,reg_lambda=0)
naive_bayes = GaussianNB()
svm = SVC()
sgd = SGDClassifier(loss="log_loss", max_iter=1000, penalty="l2")
svm = SGDClassifier(loss="modified_huber", max_iter=1000)
lgbm = LGBMClassifier()

In [ ]:
model = xgmodel

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred_test_proba = model.predict_proba(X_test)[:,1]
y_pred_train_proba = model.predict_proba(X_train)[:,1]

### Score (Accuracy, Recall, Precision, F1score, ROC, AUC)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
metrics = {
    'roc_auc_score': roc_auc_score,
    #'accuracy_score': accuracy_score,
    #'precision_score': precision_score,
    #'recall_score': recall_score,
    #f1_score': f1_score
}

In [ ]:
# SGD
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
# SVM
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
# RF
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
# RF min_samples_leaf=5, min_samples_split=10, max_depth=5
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
# RF min_samples_leaf=5, max_depth=10
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
# XGBoost
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
feature_importance = {f:i for f, i in zip(model.feature_names_in_, model.feature_importances_)}
feature_importance = {f:i/sum(i for i in feature_importance.values()) for f,i in feature_importance.items()}
feature_importance = dict(sorted(feature_importance.items(), key=lambda item: item[1], reverse=True))
feature_importance

In [ ]:
# XGBoost eta=0.3, gamma=0.2, max_depth=5, subsample=0.5,reg_alpha=10,reg_lambda=0
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

In [ ]:
# Naive-Bayes
for name, metric in metrics.items():
    print(f"Test {name}: {metric(y_test, y_pred_test_proba)}")
    print(f"Train {name}: {metric(y_train, y_pred_train_proba)}")

## Random Forest seems to get the best results so far
Let's do a grid search to look for the best parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
#iris = datasets.load_iris()
parameters = {'min_samples_leaf':(1,2,3,5,10,20), 'max_depth':(1,2,3,5,10,20)}
rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters, scoring='roc_auc')
clf.fit(X, y)

In [ ]:
clf.best_params_

In [ ]:
roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]), roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])

## XGBoost Grid Search
Let's do a grid search to look for the best parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'gamma':(0,1,10), 'max_depth':(1,2,5,10), 'subsample':(1,0.5), 'reg_alpha':(0,1,10), 'reg_lambda':(0,1,10)}
xgb = XGBClassifier()
clf = GridSearchCV(xgb, parameters, scoring='roc_auc')
clf.fit(X, y)

In [ ]:
clf.best_params_

In [ ]:
roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]), roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])


In [ ]:
%matplotlib inline
%matplotlib notebook
df_raw[['estimated_age','edad']].plot(x='estimated_age', y='edad',style='o',alpha=0.05)

In [ ]:
baja_corr = df_ready.corr().bajadeasistencia

In [ ]:
baja_corr.foreigner# abs().sort_values().head(20)

In [ ]:
%matplotlib inline
%matplotlib notebook
df_ready[df_ready.bajadeasistencia==0].estimated_age.hist(alpha=0.5, density=True)
df_ready[df_ready.bajadeasistencia==1].estimated_age.hist(alpha=0.5, density=True)

In [ ]:
df_ready.groupby('bajadeasistencia').mean()

In [ ]:
y_pred = model.predict(X_test)
y_pred_proba = np.array([p[1] for p in model.predict_proba(X_test)])

In [ ]:
dict('a':10)